# Amazon SageMaker で Keras のモデルを学習する

このサンプルノートブックでは、Amazon SageMaker の学習ジョブ機能を使って、Keras の固定長の時系列データの分類モデルを学習します。

## 前提条件
- このサンプルノートブックは Amazon SageMaker Studio の Python 3 (Tensorflow 2.6 Python 3.8 CPU) カーネルで動作を確認しています
- このサンプルノートブックは、[Keras のサンプルコード](https://github.com/keras-team/keras-io/blob/master/examples/timeseries/timeseries_classification_from_scratch.py) をベースに作成したスクリプトを使用しています。


以下のセルを実行して SageMaker Python SDK のバージョンを最新にします。

In [ ]:
!pip install -U sagemaker

以下のセルの `user_name` の文字列を、自分の名前（アルファベットと数字）に変更してから実行してください。

In [ ]:
import sagemaker
from tensorflow import keras
import numpy as np
import os
import matplotlib.pyplot as plt

sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()

project_name = 'sagemaker-keras'
user_name = 'demo'
prefix = os.path.join(project_name, user_name)

## データの準備

SageMaker の学習ジョブを利用するには、学習データが S3 に保存されている必要があります。このサンプルノートブックでは、[UCR アーカイブで公開されているデータ](http://www.j-wichard.de/publications/FordPaper.pdf) ら作成したCSV ファイルを学習データにします。

以下のセルではデータの前処理をしています。前処理の内容は、[Keras のサンプルコード](https://github.com/keras-team/keras-io/blob/master/examples/timeseries/timeseries_classification_from_scratch.py) のものをそのまま使用しています。

In [ ]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)

In [ ]:
root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

In [ ]:
classes = np.unique(np.concatenate((y_train, y_test), axis=0))

plt.figure()
for c in classes:
    c_x_train = x_train[y_train == c]
    plt.plot(c_x_train[0], label="class " + str(c))
plt.legend(loc="best")
plt.show()
plt.close()

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
num_classes = len(np.unique(y_train))

In [ ]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [ ]:
!mkdir data
np.savetxt('data/x_train.csv', x_train.squeeze())
np.savetxt('data/y_train.csv', y_train)
np.savetxt('data/x_test.csv', x_test.squeeze())
np.savetxt('data/y_test.csv', y_test)
input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix=prefix)
print('Training data is uploaded to: {}'.format(input_data))

## SageMaker の学習ジョブを使ってモデルを学習

ここからは、準備したデータを使ってモデルを学習します。モデルの学習はノートブックから直接実行することもできますが、SageMaker の学習ジョブ機能を使うと、学習ジョブ用のインスタンスを別途起動してその上でモデルの学習を実行することができます。そのため、ノートブックはスペックの低い（価格が安い）インスタンスで実行し、コンピュートリソースを多く必要とするモデルの学習はスペックの高いインスタンスで実行する、という使い分けが可能です。

Keras を使う場合は、[Tensorflow 用の Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) 使用します。使用する Tensorflow のバージョンを Estimator のパラメタで指定すると、対応するコンテナイメージを SageMaker が取得して学習で使用します。SageMaker が用意しているコンテナイメージの一覧は [こちら](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#prior-sagemaker-framework-container-versions) で公開されています。

自前のコンテナイメージを使いたい場合は [こちらのドキュメント](https://sagemaker-examples.readthedocs.io/en/latest/advanced_functionality/tensorflow_bring_your_own/tensorflow_bring_your_own.html) を参照してください。基本的な手順としては、SageMaker Training Toolkit を入れて作成したコンテナイメージを Amazon ECR リポジトリに push し、push したイメージの URI を Estimator の image_uri パラメタにセットします。

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

role = get_execution_role()
estimator = TensorFlow(
    entry_point="train.py",
    source_dir='code',
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    framework_version="2.8.0",
    py_version='py39',
    hyperparameters={'batch-size': 32,
                     'num-classes': 2,
                     'epochs': 1})

estimator.fit(input_data)

In [ ]:
trained_model = estimator.model_data

学習ジョブが完了すると、学習済みモデルは SageMaker によって model.tar.gz というファイルに圧縮されて S3 に自動的にアップロードされます。以下のセルを実行して、アップロードされた model.tar.gz をダウンロードして解凍します。

In [ ]:
!aws s3 cp $trained_model ./
!tar zvxf model.tar.gz

展開されたモデルを使って Keras の API を使って推論を実行します。問題なく推論を実行できたはずです。

In [ ]:
model = keras.models.load_model("model/1")
model.predict(x_test)

## Estimator を使って SageMaker 推論エンドポイントを起動する

上のセルで試した通り、学習したモデルは Keras を使える環境であればどこででも実行することができます。モデルを推論エンドポイントにデプロイしてリアルタイム推論で利用する場合に、SageMaker の推論エンドポイント機能が便利です。

以下のように、学習に使用した Estimator を使って、SageMaker のリアルタイム推論エンドポイントを起動することができます。Serving の詳細は [こちらのドキュメント](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html) をご参照ください。

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge'
)

推論エンドポイントを使って推論を実行します。

In [ ]:
predictor.predict(x_test)

推論エンドポイントは、稼働している間課金が続くため、不要になったら忘れずに削除してください。

In [ ]:
predictor.delete_endpoint()